In [ ]:
import pandas as pd

[Fonte](https://pandas.pydata.org/docs/getting_started/intro_tutorials/07_reshape_table_layout.html#how-to-reshape-the-layout-of-tables)

In [ ]:
titanic = pd.read_csv("data/titanic.csv")
titanic.head()

In [ ]:
air_quality = pd.read_csv("data/air_quality_long.csv", index_col="date.utc", parse_dates=True)
air_quality.head()

## Classificar linhas da tabela

Quero classificar os dados do Titanic de acordo com a idade dos passageiros.

In [ ]:
titanic.sort_values(by="Age").head()

Quero classificar os dados do Titanic de acordo com a classe da cabine e a idade em ordem decrescente.

In [ ]:
titanic.sort_values(by=['Pclass', 'Age'], ascending=False).head()


Com `DataFrame.sort_values()`, as linhas da tabela são classificadas de acordo com a(s) coluna(s) definida(s). O índice seguirá a ordem das linhas.

> Mais detalhes sobre classificação de tabelas são fornecidos na seção do guia do usuário sobre [classificação de dados](https://pandas.pydata.org/docs/user_guide/basics.html#basics-sorting).

# Formato de tabela longo a largo

Vamos usar um pequeno subconjunto do conjunto de dados de qualidade do ar. Nós nos concentramos em 
***no2*** dados e usar apenas as duas primeiras medições de cada local (ou seja, a cabeça de cada grupo). O subconjunto de dados será chamado no2_subset.

In [ ]:
# filtro apenas para dados no2
no2 = air_quality[air_quality["parameter"] == "no2"]
no2

In [ ]:
# Use 2 medidas (head(2)) para cada location (groupby)
no2_subset = no2.sort_index().groupby(["location"]).head(2)

no2_subset

Eu quero os valores para as três estações como colunas separadas uma ao lado da outra.

> A função [`pivot()`](https://pandas.pydata.org/docs/reference/api/pandas.pivot.html#pandas.pivot) é puramente reformular os dados: é necessário um único valor para cada combinação de índice/coluna.

In [ ]:
no2_subset.pivot(columns="location", values="value")

Como o pandas suporta a plotagem de várias colunas (consulte o [tutorial de plotagem](https://pandas.pydata.org/docs/getting_started/intro_tutorials/04_plotting.html#min-tut-04-plotting) ) pronto para uso, a conversão do formato de tabela longa para larga permite a plotagem de diferentes séries temporais ao mesmo tempo:

In [ ]:
no2.head()

In [ ]:
no2.pivot(columns="location", values="value").plot()

> Quando o parâmetro `index` não é definido, o índice existente (rótulos de linha) é usado.

> Para obter mais informações sobre [`pivot()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot.html#pandas.DataFrame.pivot), consulte a seção do guia do usuário sobre [como dinamizar objetos DataFrame](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-reshaping).

## Tabela dinâmica

Eu quero as concentrações médias para ***NO<sub>2</sub>*** e ***PM<sub>2.5</sub>*** em cada uma das estações em forma de tabela.

In [ ]:
air_quality.pivot_table(
    values="value", index="location", columns="parameter", aggfunc="mean"
)

> No caso de `pivot()`, os dados são apenas rearranjados. Quando vários valores precisam ser agregados (neste caso específico, os valores em diferentes etapas de tempo), podem ser usados `pivot_table()`, fornecendo uma função de agregação (por exemplo, média) sobre como combinar esses valores.

A tabela dinâmica é um conceito bem conhecido em software de planilhas. Quando estiver interessado nas margens de linha/coluna (**subtotais**) para cada variável, defina o parâmetro `margins` como `True`:

In [ ]:
air_quality.pivot_table(
    values="value",
    index="location",
    columns="parameter",
    aggfunc="mean",
    margins=True,
)

> Para obter mais informações sobre [`pivot_table()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table), consulte a seção do guia do usuário sobre [tabelas dinâmicas](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-pivot).

Caso você esteja se perguntando, [`pivot_table()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html#pandas.DataFrame.pivot_table) está de fato diretamente vinculado a [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby). O mesmo resultado pode ser derivado agrupando em ambos parametere location:

In [ ]:
air_quality.groupby(["parameter", "location"]).mean()


> Dê uma olhada em [`groupby()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas.DataFrame.groupby) em combinação com [`unstack()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.unstack.html#pandas.DataFrame.unstack) na seção do guia do usuário sobre [como combinar estatísticas e agrupar](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-combine-with-groupby) .

## Formato largo a longo

Partindo novamente da tabela de grande formato criada na seção anterior, adicionamos um novo índice ao `DataFrame` com [`reset_index()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html#pandas.DataFrame.reset_index).

In [ ]:
no2_pivoted = no2.pivot(columns="location", values="value").reset_index()

no2_pivoted.head()

Eu quero coletar toda as medições de qualidade do ar ***NO<sub>2</sub>*** em uma única coluna (formato longo).

> O método `pandas.melt()` do `DataFrame` converte a tabela de dados de formato largo para formato longo. Os cabeçalhos das colunas tornam-se os nomes das variáveis ​​em uma coluna recém-criada.

In [ ]:
no_2 = no2_pivoted.melt(id_vars="date.utc")

no_2.head()

A solução é a versão curta de como aplicar `pandas.melt()`. O método irá fundir todas as colunas NÃO mencionadas em `id_vars` em duas colunas: Uma coluna com os nomes dos cabeçalhos das colunas e uma coluna com os próprios valores. A última coluna recebe por padrão o nome value.

Os parâmetros passados `pandas.melt()` ​​podem ser definidos com mais detalhes:

In [ ]:
no_2 = no2_pivoted.melt(
    id_vars="date.utc",
    value_vars=["BETR801", "FR04014", "London Westminster"],
    value_name="NO_2",
    var_name="id_location",
)

no_2.head()

Os parâmetros adicionais têm os seguintes efeitos:

* `value_vars` define quais colunas devem ser fundidas

* `value_name` fornece um nome de coluna personalizado para a coluna de valores em vez do nome da coluna padrão `value`

* `var_name` fornece um nome de coluna personalizado para a coluna que coleta os nomes de cabeçalho de coluna. Caso contrário, leva o nome do índice ou um padrão `variable`

Portanto, os argumentos `value_name` e `var_name` são apenas nomes definidos pelo usuário para as duas colunas geradas. As colunas a serem fundidas são definidas por `id_vars` e `value_vars`.

> A conversão do formato largo para o formato longo com `andas.melt()` é explicada na seção do guia do usuário sobre [remodelação por melt](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-melt) .

## LEMBRAR

* A classificação por uma ou mais colunas é suportada por `sort_values`.

* A função `pivot` é puramente reestruturação dos dados, `pivot_table` suporta agregações.

* O inverso de `pivot` (formato longo para largo) é `melt` (formato largo para longo).